# Curso de Optimización (DEMAT)
## Proyecto Final - Método de optimización Adam

Rubén Pérez Palacios.

6 de Junio, 2022.

---

## Introducción

Es una extensión del descenso por gradiente estocástico, que recientemente ha sido muy usado para aplicaciones de aprendizaje profundo, visión cumputacional y procesamiento de lenguaje natural. El método es muy eficiente trabajando con un problema muy grande con muchos datos o parámetros. Requiere de menos memoria y es eficiente. Intuitivamente, es la combinación de los métodos ‘descenso por gradiente con momento’ y ‘propagación por error cuadrático medio’.

El metodo de Adam fue presentado por Diederik Kingma de OpenAI y Jimmy Ba de la Universidad de Toronto en su ICLR paper del 2015 ICLR titulado “Adam: A Method for Stochastic Optimization“. El nombre de Adam se deriva de estimación adaptativa de momentos.

## Intuición

Adam es la combinación de dos métodos de descenso por gradiente.

### Con Momentum

Este algoritmo es usado para ascelerar el metodo de descenso por gradiente al tomar en consideración el 'promedio ponderado exponencial' de los gradientes. Usando este promedio permite que el algoritmo converga más rapido a un minimo. El cual está dado por:

$$w_{t+1} = w_t - \alpha m_t,$$
$$m_{t} = \beta m_{t-1} - (1-\beta) \nabla f, m_0 = 0$$

- $w_t$ promedio ponderado exponencial de los gradientes en el tiempo $t$.
- $m_t$ momento en el tiempo $t$.
- $\beta$ tasa de descrecimiento del promedio de los gradientes (constante, valor cercano a 1).
- $\alpha$ tasa de aprendizaje (tamaño de paso).
- $f$ funcion a optimizar.

### Propagación por error cuadrático medio

Este algoritmo usa el promedio del decremiento de los gradientes en la adaptación del tamaño de paso. El uso del promedio de decrecimiento movil permite que el algoritmo olvide los primeros gradientes y se enfoque solamente en los mas recientes. El cúal está dado por:

$$w_{t+1} = w_t - \frac{\alpha}{(v_t+\varepsilon)^{1/2}} \nabla f,$$
$$v_{t} = \beta v_{t-1} - (1-\beta) (\nabla f)^2, v_0 = 0$$

- $w_t$ promedio ponderado exponencial del cuadrado de los gradientes en el tiempo $t$.
- $v_t$ segundo momento en el tiempo $t$.
- $\beta$ tasa de descrecimiento del promedio de los gradientes (constante, valor cercano a 1).
- $\alpha$ tasa de aprendizaje (tamaño de paso).
- $\varepsilon$ pequeña constante positiva para evitar dividir entre 0.
- $f$ funcion a optimizar.

### Adam

Uniendo estos dos metodos obtenemos

$$m_{t} = \beta_1 m_{t-1} - (1-\beta_1) \nabla f, v_{t} = \beta_2 v_{t-1} - (1-\beta_2) (\nabla f)^2$$

- $m_t$ momento en el tiempo $t$.
- $v_t$ segundo momento en el tiempo $t$.
- $\beta_1,\beta_2$ tasas de descrecimiento del promedio de los gradientes de los metodos ya mencionados (constantes, valores cercanos a 1).
- $\alpha$ tasa de aprendizaje (tamaño de paso).
- $\varepsilon$ pequeña constante positiva para evitar dividir entre 0.
- $f$ funcion a optimizar.

Ahora puesto que tanto $m_0 = 0$ como $v_0 = 0$ y $\beta_1 \sim 1$ como $\beta_2 \sim 1$, entonces $m$ y $v$ estarán viciados hacia 0. Entonces veamos que por definición tenemos que

$$v_{t} = \beta_2 v_{t-1} - (1-\beta_2) (\nabla f)^2 = (1-\beta_2)\sum_{i=1}^t\beta_2^{t-i}\cdot g_i^2,$$

por lo que con linealidad de la esperanza obtenemos

$$E[v_{t}] = E[(1-\beta_2)\sum_{i=1}^t\beta_2^{t-i}\cdot g_i^2] = E[\sum_{i=1}^t(1-\beta_2^t)\cdot g_i^2] = (1-\beta_2^t)E[g_i\cdot g_i],$$

por lo tanto un estmiador incesagado del segundo momento no centralizado de los gradientes es

$$\hat{v}_t = \frac{v_t}{1-\beta_1},$$

analogamente un estmiador incesagado del primer momento de los gradientes es

$$\hat{m}_t = \frac{m_t}{1-\beta_1}.$$

Finalmente obtenemos que

$$w_{t+1} = w_t - \alpha \frac{\hat{m}_t}{\sqrt{\hat{v}_t}+\varepsilon}.$$

Recordando las fortalezas de los algortimos descritos anteriormente, podemos concluir que Adam es un método de optimización de descenso por gradiente, que toma a considerención el primer y segundo momento de los gradientes. Lo cual le permite acercarse de manera más rapido a un minimo, dar pasos sufcientemente grandes para sobrepasar mínimos locales y controlando la osiclacion cuando este alcanza un mínimo global.


In [135]:
import numpy as np
from numpy.random import rand
from numpy.random import seed
import itertools
import plotly.graph_objects as go
from IPython.display import Image

## Implementación del método de optimicación de Adam

In [136]:
def adam(f, gf, x0, maxN, alpha, beta1, beta2, tol, eps=1e-8):
	print("Adam")
	solutions = list()
	x_k = x0.copy()
	solutions.append(x_k)
	
	# Conidcion de inicio del primer y segundo momento
	m_k = np.zeros(x0.shape[1])
	v_k = np.zeros(x0.shape[1])
	k = 0
	while (k < maxN):
		
		gf_k = gf(x_k)

		if np.linalg.norm(gf_k) < tol:
			break
		
		# Calcular siguiente punto

		# m(t) = beta1 * m(t-1) + (1 - beta1) * g(t)
		m_k = beta1 * m_k + (1.0 - beta1) * gf_k
		# v(t) = beta2 * v(t-1) + (1 - beta2) * g(t)^2
		v_k = beta2 * v_k + (1.0 - beta2) * (gf_k * gf_k)
		# mhat(t) = m(t) / (1 - beta1(t))
		mhat_k = m_k / (1.0 - beta1**(k+1))
		# vhat(t) = v(t) / (1 - beta2(t))
		vhat_k = v_k / (1.0 - beta2**(k+1))
		# x(t) = x(t-1) - alpha * mhat(t) / (sqrt(vhat(t)) + ep)
		x_k = x_k - alpha * mhat_k / (np.sqrt(vhat_k) + eps)
		
		solutions.append(x_k.copy())
		
		k = k + 1
		# print('>%d f(%s) = %.5f' % (k, x_k, f(x_k)))
		
	return solutions

## Implementación de otros metodos de optimicación

In [137]:
from scipy.linalg import cho_solve, cho_factor

def backtracking(f, f_k, gf_k, x_k, p_k, alpha, rho, c):
    while (f(x_k + alpha*p_k) > f_k + c*alpha*gf_k@p_k.T):
        alpha = rho*alpha
    return alpha

def gradient_descent(f, gf, x_0, alpha, rho, c, maxN, tol):
    print("Gradient")
    x_k = x_0.copy()
    points = list()
    points.append(x_k.copy())
    k = 0
    while (k < maxN):
        gf_k = gf(x_k)
        if np.linalg.norm(gf_k) < tol:
            res = 1
            break
        p_k = -gf_k
        a_k = backtracking(f, f(x_k), gf_k, x_k, p_k, alpha, rho, c)
        x_k = x_k + a_k*p_k
        points.append(x_k.copy())
        k = k + 1
        #print('>%d f(%s) = %.5f' % (k, x_k, f(x_k)))
    return points

def newthon_method_backtracking(f, gf, Hf, x_0, alpha, rho, c, maxN, tol):
    print("Newthon")
    x_k = x_0.copy()
    k = 0
    points = list()
    points.append(x_k.copy())
    while (k < maxN):
        gf_k = gf(x_k)
        if np.linalg.norm(gf_k) < tol:
            break
        Hf_k = Hf(x_k)
        try:
            p_k = cho_solve(cho_factor(Hf_k), -gf_k)
        except:
            break
        a_k = backtracking(f, f(x_k), gf_k, x_k, p_k, alpha, rho, c)
        x_k = x_k + a_k * p_k
        points.append(x_k.copy())
        k = k + 1
        #print('>%d f(%s) = %.5f' % (k, x_k, f(x_k)))
    return points

## Comparación de metodos de optimicación

In [138]:
def find_opt(f, gf, hf, bounds, maxN, tol, x0, alpha, beta1, beta2, rho, c, f_name):
	method_names = np.array(
		[
			"Newthon",
			"Gradient Descent",
			"Adam",
		]
	)
	solutions = np.array(
		[
			np.array(newthon_method_backtracking(f, gf, hf, x0, alpha, rho, c, maxN, tol)),
			np.array(gradient_descent(f, gf, x0, alpha, rho, c, maxN, tol)),
			np.array(adam(f, gf, x0, maxN, alpha, beta1, beta2, tol)),
		]
	)

	t = 100
	x = np.linspace(bounds[0,0], bounds[0,1], t)
	y = np.linspace(bounds[1,0], bounds[1,1], t)
	fig = go.Figure()
	fig.add_surface(
		x = x,
		y = y,
		z = np.array([f(np.array([[x_i,y_i]])) for y_i, x_i in itertools.product(y,x)]).reshape(t,t),
		name = 'f'
	)
	for solution, name in zip(solutions, method_names):
		fig.add_trace(
			go.Scatter3d(
				x = solution[:,0,0],
				y = solution[:,0,1],
				z = f(np.array([[solution[:,0,0],solution[:,0,1]]])),
				name = name
			)
		)
	fig.update_layout(
			template="simple_white",
			title=f_name,
			width=1000,
			height=1000
	)
	fig.show()

	fig = go.Figure()
	fig.add_contour(
		x=x,
		y=y,
		z=np.array([f(np.array([[x_i,y_i]])) for y_i, x_i in itertools.product(y,x)]).reshape(t,t),
		name='f'
	)
	for solution, name in zip(solutions, method_names):
		fig.add_trace(
			go.Scatter(
				x = solution[:,0,0],
				y = solution[:,0,1],
				name = name
			)
		)
	fig.update_layout(
			template="simple_white",
			title=f_name,
			width=1000,
			height=1000
	)
	fig.show()

## Ejemplo con función de la esfera en $R^2$

In [139]:
def f_paraboloid(x):
	return x[0,0]**2 + x[0,1]**2
 
def gf_paraboloid(x):
	return np.array(
		[
			2*x[0,0],
			2*x[0,1]
		]
	)

def hf_paraboloid(x):
    return np.array(
        [
            [
                2,
                0
            ],
            [
                0,
                2
            ]
        ]
    )

seed(1)
bounds = np.array([[-1.0, 1.0], [-1.0, 1.0]])
maxN = 60
alpha = 0.02
beta1 = 0.8
beta2 = 0.999
x0 = np.array([bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])])
tol = np.finfo(float).eps**(1/2)
rho = 0.8
c =  0.0001

find_opt(f_paraboloid, gf_paraboloid, hf_paraboloid, bounds, maxN, tol, x0, alpha, beta1, beta2, rho, c, "Función de la Esfera")

Newthon
Gradient
Adam


## Ejemplo con función de Rosenbrock en $R^2$

In [140]:
def f_rosenbrock(x):
    return 100*(x[0,1]-x[0,0]**2)**2 + (1-x[0,0])**2
    
def gf_rosenbrock(x):
    return np.array(
        [
            400*x[0,0]**3-400*x[0,0]*x[0,1]+2*x[0,0]-2,
            200*(x[0,1]-x[0,0]**2)
        ]
    )

def hf_rosenbrock(x):
    return np.array(
        [
            [
                1200*x[0,0]**2-4*x[0,1]+2,
                -400*x[0,0]
            ],
            [
                -400*x[0,0],
                200
            ]
        ]
    )

seed(1)
bounds = np.array([[-1.0, 1.0], [-1.0, 1.0]])
maxN = 1000
alpha = 0.02
beta1 = 0.8
beta2 = 0.999
x0 = np.array([bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])])
tol = np.finfo(float).eps**(1/2)
rho = 0.8
c =  0.0001

find_opt(f_rosenbrock, gf_rosenbrock, hf_rosenbrock, bounds, maxN, tol, x0, alpha, beta1, beta2, rho, c, "Funcion de Rosenbrock")

Newthon
Gradient
Adam


## Ejemplo con función de Rastring en $R^2$

In [141]:
def f_rastring(x):
	return 10*2 + x[0,0]*x[0,0]- 10*np.cos(2*np.pi*x[0,0]) + x[0,1]*x[0,1]- 10*np.cos(2*np.pi*x[0,1])
 
def gf_rastring(x):
	return np.array(
		[
            2*x[0,0]+2*np.pi*10*np.sin(2*np.pi*x[0,0]),
            2*x[0,1]+2*np.pi*10*np.sin(2*np.pi*x[0,1])
        ]
	)

def hf_rastring(x):
    return np.array(
        [
            [
                2 + (2*np.pi)**2*10*np.cos(x[0,0]),
                0
            ],
            [
                0,
                2 + (2*np.pi)**2*10*np.cos(x[0,1])
            ]
        ]
    )
    
seed(1)
bounds = np.array([[-3.15, 3.15], [-3.15, 3.15]])
maxN = 1000
alpha = 0.2
beta1 = 0.9
beta2 = 0.999
x0 = np.array([[-1.4,-1.4]])
tol = np.finfo(float).eps**(1/2)
rho = 0.8
c =  0.0001

find_opt(f_rastring, gf_rastring, hf_rastring, bounds, maxN, tol, x0, alpha, beta1, beta2, rho, c, "Función de Rastring")

Newthon
Gradient
Adam


C:\Users\batma\AppData\Local\Temp\ipykernel_3884\3143451642.py:9: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



## Conclusión

Podemos ver como en el ejemplo de la función de la circunferencia en $R^2$ el método de Adam alcanza más rapido el mínimo global que los otros dos métodos de optimización. En el ejemplo de la función Rosenbrock con pocas iteraciones el descenso por gradiente esta más cerca del minimo pero mientras las iteraciones aumentan el de Adam lo alcanza y lo pasa. Por último en la función de Rastring podemos ver como el descenso por gradiente avanza cuantas más iteraciones le indiquemos, el método de Newton se estanca en un mínimo local, y en cambio el método de Adam logra pasar este mínimo local y se acerca cada vez más al mínimo global.

Lo cúal ejemplifica las dos fortalezas de este método. sumado a sus bondades de manejo de memoria, para casos multivariados muy grandes puede ser un algoritmo ejemplar para optimizar.